In [42]:
# imports
import pretty_midi
import warnings
import os
import sys
import re

In [1]:
temp = (2, 3, 52, 5, 35, 5, 3, 5)

In [2]:
['<s>'] * 8 + ['<{}-{}>'.format(j, k) for j, k in enumerate(temp)]

['<s>',
 '<s>',
 '<s>',
 '<s>',
 '<s>',
 '<s>',
 '<s>',
 '<s>',
 '<0-2>',
 '<1-3>',
 '<2-52>',
 '<3-5>',
 '<4-35>',
 '<5-5>',
 '<6-3>',
 '<7-5>']

### Example of MIDI file with empty tracks

In [3]:
black_or_white = pretty_midi.PrettyMIDI('/Users/huongngo/Desktop/RESEARCH/text-to-midi/captioning/examples/Black or White copy.mid')

/Users/huongngo/opt/anaconda3/envs/gehirn/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [4]:
for instrument in black_or_white.instruments:
    print(instrument.notes[:2])

[Note(start=18.873909, end=19.186408, pitch=71, velocity=110), Note(start=19.132529, end=19.471968, pitch=76, velocity=114)]
[Note(start=2.068964, end=2.074352, pitch=36, velocity=111), Note(start=6.460125, end=6.465513, pitch=36, velocity=89)]
[Note(start=2.068964, end=2.370688, pitch=28, velocity=116), Note(start=10.086200, end=10.355596, pitch=40, velocity=125)]
[Note(start=35.167000, end=37.235964, pitch=66, velocity=70), Note(start=35.167000, end=37.235964, pitch=71, velocity=70)]
[Note(start=2.613145, end=2.726291, pitch=69, velocity=116), Note(start=2.607757, end=2.737067, pitch=64, velocity=107)]
[Note(start=93.103380, end=93.146483, pitch=40, velocity=91), Note(start=93.189587, end=93.324285, pitch=40, velocity=95)]
[Note(start=46.842638, end=47.058155, pitch=40, velocity=95), Note(start=46.842638, end=47.074319, pitch=45, velocity=101)]
[Note(start=103.458976, end=103.593674, pitch=68, velocity=87), Note(start=103.458976, end=103.604450, pitch=79, velocity=87)]
[Note(start=48

In [5]:
black_or_white.key_signature_changes

[]

In [6]:
black_or_white.get_tempo_changes()

(array([0.]), array([116.00008507]))

A warning is returned stating that there is a tempo, key or time signature event on one of the tracks after the first one. Here, track is not referred to the MIDI file tracks, but the pretty_midi tracks. The first track always contains the metadata of the song (key signature, time signature etc.). It seems like there is an empty track when there is a tempo, key or time signature event on one of the tracks after the first one. 

The empty track can't be removed but we can flag and ignore that track. It can only be manually modified. 

### Example of same MIDI file that has been modified to not have empty tracks

In [7]:
black_or_white_m = pretty_midi.PrettyMIDI('/Users/huongngo/Desktop/RESEARCH/text-to-midi/captioning/examples/black_or_white_m.mid')

In [8]:
for instrument in black_or_white.instruments:
    print(instrument.notes[:2])

[Note(start=18.873909, end=19.186408, pitch=71, velocity=110), Note(start=19.132529, end=19.471968, pitch=76, velocity=114)]
[Note(start=2.068964, end=2.074352, pitch=36, velocity=111), Note(start=6.460125, end=6.465513, pitch=36, velocity=89)]
[Note(start=2.068964, end=2.370688, pitch=28, velocity=116), Note(start=10.086200, end=10.355596, pitch=40, velocity=125)]
[Note(start=35.167000, end=37.235964, pitch=66, velocity=70), Note(start=35.167000, end=37.235964, pitch=71, velocity=70)]
[Note(start=2.613145, end=2.726291, pitch=69, velocity=116), Note(start=2.607757, end=2.737067, pitch=64, velocity=107)]
[Note(start=93.103380, end=93.146483, pitch=40, velocity=91), Note(start=93.189587, end=93.324285, pitch=40, velocity=95)]
[Note(start=46.842638, end=47.058155, pitch=40, velocity=95), Note(start=46.842638, end=47.074319, pitch=45, velocity=101)]
[Note(start=103.458976, end=103.593674, pitch=68, velocity=87), Note(start=103.458976, end=103.604450, pitch=79, velocity=87)]
[Note(start=48

In [37]:
black_or_white_m.key_signature_changes

[KeySignature(key_number=0, time=0.0)]

In [9]:
pretty_midi.key_number_to_key_name(black_or_white_m.key_signature_changes[0].key_number)

'C Major'

In [40]:
black_or_white_m.get_tempo_changes()

(array([0.]), array([116.00008507]))

In [36]:
black_or_white_m.get_tempo_changes()[1][0]

116.00008506672903

### Checking the % of MIDI files with empty tracks

In [11]:
data_path = '/Users/huongngo/Desktop/RESEARCH/clean_midi'

In [43]:
def contains_number_pattern(string):
    pattern = r"\.\d\."
    return bool(re.search(pattern, string))

In [44]:
# getting number of valid files
midi_key_tempo = {} # holding the key and tempo of the midi files
total_files = 0
for root, dirs, files in os.walk(data_path, topdown = False):
    for f in files:
        if f == 'out.txt' or f == '.DS_Store' or f == 'err.txt': # ignoring misc files
            continue
        # if len(f.split('.')) == 3 and 'feat' not in f: # to not process multiple copies of the file (ignoring files that have feat. because should still consider those)
        #     continue
        # if len(f.split('.')) > 3: # if there's multiple versions of songs with feat. (the split will result in list > 3)
        #     continue
        if contains_number_pattern(f):
            continue
        midi_key_tempo[f] = (0, 0) # (key, tempo)
        total_files += 1
print(total_files)

10309


In [ ]:
empty_track_files = []
corrupted_files = []
other_files = []
files_processed = 0
for root, dirs, files in os.walk(data_path, topdown = False):
    for f in files:
        if f == 'out.txt' or f == '.DS_Store' or f == 'err.txt': # ignoring misc files
            continue
        if len(f.split('.')) == 3 and 'feat' not in f: # to not process multiple copies of the file (ignoring files that have feat. because should still consider those)
            continue
        if len(f.split('.')) > 3: # if there's multiple versions of songs with feat. (the split will result in list > 3)
            continue
        midi_path = os.path.join(root, f) # path to midi file
        with warnings.catch_warnings(record=True) as w: # catching warnings
            # cause all warnings to always be triggered
            warnings.simplefilter("always")
            try:
                # trigger a warning
                midi = pretty_midi.PrettyMIDI(midi_path)
                # Verify some things
                if len(w) == 0: # no warnings -> Not empty track and no issues
                    assert len(midi.key_signature_changes) == 1
                    key = pretty_midi.key_number_to_key_name(midi.key_signature_changes[0].key_number)
                    assert len(midi.get_tempo_changes()) == 2
                    tempo = midi.get_tempo_changes()[1][0]
                    midi_key_tempo[f] = (key, tempo) # inserting key and tempo
                elif len(w) == 1: # Runtime Warning detected -> Empty track
                    warning = w[0]
                    assert issubclass(warning.category, RuntimeWarning)
                    print('')
                    print(f'Warning detected for file: {midi_path}')
                    print(f'Warning: {warning.message}')
                    empty_track_files.append(midi_path)
                    print(f'Percentage of MIDI files w/ empty tracks: {len(empty_track_files) * 100 / total_files:.2f}%')
                    print('')
                elif len(w) > 0: # More than one warning -> Not sure what is issue, delegate to other_files for now
                    print('')
                    print(f'Warning detected for file: {midi_path}')
                    print(f'Warning: {warning.message}')
                    other_files.append(midi_path)
                    print('')
            except Exception as e: # catching the error
                print('')
                print(f'An exception of type {type(e).__name__} detected for file: {midi_path}')
                corrupted_files.append(midi_path)
                print(f'Percentage of MIDI files that have error: {len(corrupted_files) * 100 / total_files:.2f}%')
                print('')
        files_processed += 1
        print(f'Progress: {files_processed * 100 / total_files:.2f}%')

In [29]:
empty_track_files[:5]

['/Users/huongngo/Desktop/RESEARCH/clean_midi/The Corrs/Runaway.mid',
 '/Users/huongngo/Desktop/RESEARCH/clean_midi/Cocciante/Io canto.mid',
 '/Users/huongngo/Desktop/RESEARCH/clean_midi/Cocciante/Parole sante, zia Lucia.mid',
 '/Users/huongngo/Desktop/RESEARCH/clean_midi/Amos, Tori/Raspberry Swirl.mid',
 '/Users/huongngo/Desktop/RESEARCH/clean_midi/Amos, Tori/Concertina.mid']

In [33]:
# percentage of empty track files
len(empty_track_files) * 100 / total_files

7.778317938745746

In [30]:
corrupted_files[:5]

['/Users/huongngo/Desktop/RESEARCH/clean_midi/Cocciante/Ammassati e distanti.mid',
 '/Users/huongngo/Desktop/RESEARCH/clean_midi/Amos, Tori/The Wrong Band.mid',
 '/Users/huongngo/Desktop/RESEARCH/clean_midi/Metallica/(Anesthesia)-Pulling Teeth.mid',
 '/Users/huongngo/Desktop/RESEARCH/clean_midi/The Cranberries/Animal Instinct.mid',
 '/Users/huongngo/Desktop/RESEARCH/clean_midi/Bee Gees/Too Much Heaven.mid']

In [34]:
# percentage of corrupted files
len(corrupted_files) * 100 / total_files

1.4681575109382596

In [31]:
other_files

[]